# How to get data from SNB BNS

This is a sample notebook that demonstrates how to access data from SNB BNS web site. 

### Background

The download URL for the Excel is not directly avaialble from the web page.  We can, however, extract the URL using the following procedure.

### Example

Find a chart on the web site e.g. https://data.snb.ch/en/topics/ziredev

Click on the Download Excel link at the top. Notice that the URL changes to 
https://data.snb.ch/en/topics/ziredev#!/chart/rendeidglfzch

This URL reveals the chartId (`rendeidglfzch`) that is required for locating the fileId for the Excel download file.

### Procedure

**Step 1.** Request the fileid using the `getChartExcelFileRef` endpoint.

```
URL:
https://data.snb.ch/json/chart/getChartExcelFileRef?lang=EN&pageViewTime=20180408_050806

Method:
POST

Body:
{"chartId":"rendeidglfzch"}
```

**Step 2.** Extract the fileid from the response of the above request.  Note that the text looks corrupted at the beginning.  We do not care about that, however, since we will be stripping them off to obtain the fileid.

```
Response:
)]}\',\n{"fileid":"a80cb2bd79caab4dc7ac831a195a52320d2ab822c89819e8a405eb9c9937ed0c"}

What we need:
a80cb2bd79caab4dc7ac831a195a52320d2ab822c89819e8a405eb9c9937ed0c
```

**Step 3.** Download the Excel file via Pandas' `read_excel` function.  The URL is constructed with the fileid as follows:

```
URL:
'https://data.snb.ch/json/table/getFile?fileId=a80cb2bd79caab4dc7ac831a195a52320d2ab822c89819e8a405eb9c9937ed0c&pageViewTime=20180408_043419&lang=en'
```

In [1]:
import pandas as pd
import datetime
import requests
import re

In [2]:
# get current timestamp
dt = f"{datetime.datetime.utcnow():%Y%m%d_%H%M%S}"
dt

'20180408_052651'

In [3]:
# construct reference file id URL
refurl = "https://data.snb.ch/json/chart/getChartExcelFileRef?lang=EN&pageViewTime=%s" % dt
refurl

'https://data.snb.ch/json/chart/getChartExcelFileRef?lang=EN&pageViewTime=20180408_052651'

In [4]:
# Get file id for this chart 
x = requests.post(refurl, headers={"Content-type": "application/json;charset=UTF-8"}, data='{"chartId":"rendeidglfzch"}')

In [5]:
# Convert the response to String type
s = x.content.decode("UTF-8")
s

')]}\',\n{"fileid":"a80cb2bd79caab4dc7ac831a195a52320d2ab822c89819e8a405eb9c9937ed0c"}'

In [6]:
# Strip away the beginning and the end of the string, leaving the fileid
s = re.sub(r'^.*\n{"fileid":"', '', s)
s = re.sub(r'"}', '', s)
s

'a80cb2bd79caab4dc7ac831a195a52320d2ab822c89819e8a405eb9c9937ed0c'

In [7]:
# Construct download URL
fileurl = 'https://data.snb.ch/json/table/getFile?fileId=%s&pageViewTime=%s&lang=en' % (s, dt)
fileurl

'https://data.snb.ch/json/table/getFile?fileId=a80cb2bd79caab4dc7ac831a195a52320d2ab822c89819e8a405eb9c9937ed0c&pageViewTime=20180408_052651&lang=en'

In [8]:
# Read file directly.  Skip unnecessary rows at the top.
df = pd.read_excel(fileurl, skiprows=15)

In [9]:
# Let's take a look!
df

,Unnamed: 0,2 years,5 years,10 years,20 years
0,1988-01-04,3.691,3.823,4.015,4.229
1,1988-01-05,3.691,3.823,4.015,4.229
2,1988-01-06,3.691,3.823,4.015,4.229
3,1988-01-07,3.691,3.823,4.015,4.229
4,1988-01-08,3.691,3.823,4.015,4.229
5,1988-01-11,3.720,3.847,4.015,4.232
6,1988-01-12,3.720,3.847,4.015,4.232
7,1988-01-13,3.720,3.847,4.015,4.232
8,1988-01-14,3.720,3.847,4.015,4.232
9,1988-01-15,3.720,3.847,4.015,4.232
